In [ ]:
from src.agents import Agent
from src.utils import normalize, extract_walls_from_geometry
from shapely.geometry import Polygon, Point
import numpy as np
import pandas as pd
import pedpy
import jupedsim as jps
from jupedsim.internal.notebook_utils import animate

In [ ]:
outer = [(0, 0), (10, 0), (10, 10), (0, 10)]
hole = [(4, 4), (6, 4), (6, 6), (4, 6)]
geometry = Polygon(shell=outer, holes=[hole])
distribution  = [(7, 0), (10, 0), (10, 10), (7, 10)]
walls = extract_walls_from_geometry(geometry)
exits = [Polygon([(0, 0), (2, 0), (2, 2), (0, 2)])]
positions = jps.distribute_by_number(
    polygon=Polygon(distribution),
    number_of_agents=10,
    distance_to_agents=0.4,
    distance_to_polygon=0.2)

agents = []
for positions in positions:
    agent = Agent(position=positions, velocity=[-1.0, 0.0])
    agents.append(agent)

h_i = np.array([0.0, 0.0])
dt, steps = 0.1, 200

active_agents = list(range(len(agents)))  # keep track of active agent indices

# Simulation
trajectories = []
for frame in range(steps):
    # Store next state separately to avoid updates affecting other agents' inputs
    next_positions = []
    next_velocities = []
    #for agent1 in agents: calculate the influnce of all other agents
    print (f"{frame = }", end="\r")
    for i in active_agents:
        agent = agents[i]
        # Gather others (positions and velocities)
        others = [(agents[j].x, agents[j].v) for j in active_agents if j != i]
        
        # Compute forces and update
        agent.compute_forces(others, walls, [], [], exits, h_i)
        agent.update(dt)
        
        o = normalize(agent.v)
        trajectories.append({
            "frame": frame,
            "id": i,
            "x": agent.x[0],
            "y": agent.x[1],
            "ox": o[0],
            "oy": o[1]
        })
    
    # Remove agents who have reached the exit
    still_active = []
    for i in active_agents:
        if not Point(agents[i].x).within(exits[0]):
            still_active.append(i)
        else:
            print(f"Agent {i} exited at frame {frame}", end="\r")
    
    active_agents = still_active
    
    # Stop early if all agents are done
    if not active_agents:
        break
# Convert to DataFrame
df = pd.DataFrame(trajectories)

In [ ]:

traj = pedpy.TrajectoryData(data=df, frame_rate=1/dt)
walkable_area = pedpy.WalkableArea(polygon=geometry)
pedpy.plot_trajectories(walkable_area=walkable_area, traj=traj)

In [ ]:
animate(data=traj, area=walkable_area, every_nth_frame=1, title_note="Hirai&Tarui, 1975")